In [1]:
import numpy as np
import cv2
import pickle
import glob
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from tracker import tracker

In [2]:
#read in the saved objpoints and imgpoints
dist_pickle = pickle.load( open( "camera_cal/calibration.p", "rb" ))
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]


In [3]:
def red_abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh=(0,255)):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    red = img[:,:,0]
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(red, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(red, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

In [4]:
def green_abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh=(0,255)):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    green = img[:,:,1]
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(green, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(green, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

In [5]:
def abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh=(0,255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

In [6]:
def red_abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh=(0,255)):
    # Convert to grayscale
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    red = img[:,:,2]
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(red, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(red, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return binary_output

In [7]:
# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

In [8]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

In [9]:
# Define a function that thresholds the S-channel of HLS
def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output

In [10]:
def color_threshold(image, sthresh=(0.255), vthresh=(0,255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0]) & (s_channel <= sthresh[1]) ] = 1
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= vthresh[0]) & (v_channel <= vthresh[1]) ] = 1    
    
    output = np.zeros_like(s_channel)
    output[(s_binary ==1) & (v_binary == 1)] = 1
    return output

In [11]:
def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level + 1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width)):min(int(center+width),img_ref.shape[1])]=1
    return output


In [ ]:
def process_image(img):

    #undistort the image
    img = cv2.undistort(img, mtx, dist, None, mtx)


    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)

    preprocessImage = np.zeros_like(img[:,:,0])

    red_gradx = red_abs_sobel_thresh(img, orient= 'x', thresh = (12,180))
    red_grady = red_abs_sobel_thresh(img, orient= 'y', thresh = (35,120))
    redxy = (red_gradx ==1)&(red_grady==1)

    green_gradx = green_abs_sobel_thresh(img, orient= 'x', thresh = (12,180))
    green_grady = green_abs_sobel_thresh(img, orient= 'y', thresh = (35,120))
    greenxy = (green_gradx ==1)&(green_grady==1)

    c_binary = color_threshold(img,sthresh=(110,255), vthresh=(80,255))
    #s_channel = hls_select(img, (40, 200))


    preprocessImage[(redxy & greenxy) |  (c_binary==1) ] = 255

 
    img_size = (img.shape[1], img.shape[0])


    #work on defining perspective transform
    img_size = (img.shape[1], img.shape[0])


    bot_width = .76 #percent of bottom trapezoid height
    mid_width = .08 #percent of middle trapedzoid height

    height_pct = .64 #percent of trapezoid height
    bot_trim = .935 #percent from top to bottom to avoid car hood
    offset = img_size[0]*.25
    # Source coordinates
    src = np.float32([[img.shape[1]*(.5-mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(.5+mid_width/2),img.shape[0]*height_pct], [img.shape[1]*(.5+bot_width/2),img.shape[0]*bot_trim] , [img.shape[1]*(.5-bot_width/2),img.shape[0]*bot_trim] ])
    # Destination coordinates
    dst = np.float32([[offset, 0], [img_size[0]-offset,0], [img_size[0]-offset,img_size[1]],[offset,img_size[1]]])
    
    
    #perform the transform
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preprocessImage, M, img_size, flags = cv2.INTER_LINEAR)
    
    window_width = 35
    window_height = 80
    
    #setup the overall class to do all the tracking
    curve_centers = tracker(Mywindow_width = window_width, Mywindow_height = window_height, Mymargin = 35, My_ym= 10/720, My_xm=3/384, Mysmooth_factor = 30)

    window_centroids = curve_centers.find_windows_centroids(warped)
    
    #points used to draw all the left and right points
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    #points used to find the left and right lanes
    rightx = []
    leftx = []
    
    # go through each level and draw the windows
    for level in range(0,len(window_centroids)):
        #window_mask is a function to draw window areas
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])
        
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        #add graphic points from window mask here to total pixels found
        l_points[(l_points == 255) | ((l_mask ==1 ) ) ] = 255
        r_points[(r_points == 255) | ((r_mask ==1 ) ) ] = 255
        
    #draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) #overlay the original image with window results
    
    #fit the lane boundaries to the left, right, center positions found
    yvals = range(0,warped.shape[0])
    
    res_yvals = np.arange(.9*(warped.shape[0]-(window_height/2)),0,-window_height)
    
    left_fit = np.polyfit(res_yvals, leftx,2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals +left_fit[2]
    left_fitx = np.array(left_fitx,np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx,2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals +right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)   
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    #middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    inner_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    
    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road, [left_lane], color = [255,0,0] )
    cv2.fillPoly(road, [right_lane], color = [0,0,255] )
    cv2.fillPoly(road, [inner_lane], color = [0,255,0] )
    cv2.fillPoly(road_bkg, [left_lane],color =[255,255,255] )
    cv2.fillPoly(road_bkg,[right_lane],color =[255,255,255] )
    
    road_warped = cv2.warpPerspective(road,Minv,img_size,flags=cv2.INTER_LINEAR)
    #road_warped_bkg = cv2.warpPerspective(road_bkg,Minv,img_size,flags=cv2.INTER_LINEAR)
    
    #base = cv2.addWeighted(img,1.0,road_warped_bkg,-1.0,0.0)
    result = cv2.addWeighted(img,1.0,road_warped,0.7,0.0)
    
    ym_per_pix = curve_centers.ym_per_pix #meters per pixel in y direction
    xm_per_pix = curve_centers.xm_per_pix #meters per pixel in x direction
    
    curve_fit_cr = np.polyfit(np.array(res_yvals,np.float32)*ym_per_pix, np.array(leftx,np.float32)*xm_per_pix,2)
    curverad = ((1+(2*curve_fit_cr[0]*yvals[-1]*ym_per_pix+curve_fit_cr[1])**2)**1.5)/np.absolute(2*curve_fit_cr[0])
    
    #calculate the offset of the car on the road
    camera_center = (left_fitx[-1]+right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_diff <=0:
        side_pos= 'right'
        
    #draw the text showing curvature, offset, and speed
    cv2.putText(result, 'Radius of Curvature = ' + str(round(curverad,3))+'(m)',(50,50) , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    cv2.putText(result, 'Vehicle is = ' + str(round(center_diff,3))+'m '+side_pos+' of center',(50,100) , cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255),2)
    
    return result

In [ ]:
Output_video = 'output1_tracked1.mp4'
Input_video = 'project_video.mp4'

clip1 = VideoFileClip(Input_video)
video_clip = clip1.fl_image(process_image) # NOTE: this function expects color images!!
video_clip.write_videofile(Output_video, audio = False)

[MoviePy] >>>> Building video output1_tracked1.mp4
[MoviePy] Writing video output1_tracked1.mp4


 39%|███▉      | 489/1261 [01:14<01:58,  6.52it/s]